In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import scipy.io
from sklearn.neighbors import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

### load data

In [ ]:
mat = scipy.io.loadmat('../datasets/fashion.mat')

In [ ]:
X_train = mat['Xtr']
y_train = mat['ytr']
X_test = mat['Xtst']
y_test = mat['ytst']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

### 1. nearest centroid classification with original images (no pca yet)

In [ ]:
clf = NearestCentroid()
clf.fit(X_train, y_train.ravel())

y_pred = clf.predict(X_test)

print('error = ', 1- accuracy_score(y_test,y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=clf.classes_)
disp.plot();

### 2. Apply PCA to reduce dimension first and then perform k nearest neighbor (kNN) classification (Euclidean distance, no weighting).

For this data set, 50 dimensions might no longer be the best choice. So there are two parameters to be tuned: m (number of pca dimensions) and k (number of nearest neighbors).

A large scale grid search over (m,k) could find the optimal pair in terms of test error (or validation error). However, it will be slow.

Here, let us fix m to a small number of values such as 50, 100, 150 (feel free to change these numbers). For each value of m, perform PCA to project both the training and test data into the same m dimensional space. Afterwards, perform kNN classification on the m-dimensional PCA reduced data, for k = 1, 2, ..., 12. Plot the test errors against k, one curve for each fixed value of m.

What is a good choice of the pair (m,k), in terms of test error?

In [ ]:
m_values = [50, 100, 150, 200]
k_values = range(1, 13)

errors = {}

for m in m_values:
    pca = PCA(n_components=m)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    for k in k_values:
        clf = KNeighborsClassifier(n_neighbors=k)
        clf.fit(X_train_pca, y_train.ravel())
        y_pred = clf.predict(X_test_pca)
        errors[(m, k)] = 1 - accuracy_score(y_test, y_pred)
        print('m =', m, 'k =', k, 'error =', errors[(m, k)])

best_params = min(errors, key=errors.get)
print('best_params =', best_params)
print('error =', errors[best_params])

In [ ]:
# Plot all lines for each m, k pair

for m in m_values[1:]:
    plt.plot(k_values, [errors[(m, k)] for k in k_values], label=f'm={m}')
plt.legend()
plt.xlabel('k')
plt.ylabel('error')
plt.grid()
plt.show()

In [ ]:
best_m = best_params[0]
best_m

### 3 For the best value of m found above, perform unweighted kNN classification with city block distance ($\ell_1$) for k = 1,2,..., 12. 

Plot the test errors against k. How does it compare with unweighted kNN + Euclidean metric ($\ell_2$)?

In [ ]:
k_values = range(1, 13)

manhatten_errors = []
euclidean_errors = []

pca = PCA(n_components=best_m)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
    clf.fit(X_train_pca, y_train.ravel())
    y_pred = clf.predict(X_test_pca)
    manhatten_errors.append(1 - accuracy_score(y_test, y_pred))
    print('Manhattan: m =', best_m, 'k =', k, 'error =', manhatten_errors[k-1])

    clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    clf.fit(X_train_pca, y_train.ravel())
    y_pred = clf.predict(X_test_pca)
    euclidean_errors.append(1 - accuracy_score(y_test, y_pred))
    print('Euclidean: m =', best_m, 'k =', k, 'error =', euclidean_errors[k-1])

In [ ]:
plt.plot(k_values, manhatten_errors, label='Manhatten')
plt.plot(k_values, euclidean_errors, label='Euclidean')
plt.legend()
plt.xlabel('k')
plt.ylabel('error')
plt.grid()
plt.title('Manhattan vs Euclidean')
plt.show()

### 4. For the best value of m found above, perform kNN classification + Euclidean metric, with inverse distance weights, for k = 1,2,...,12. 

Plot the test errors against k.  How does the weighted kNN compare with the unweighted kNN (both with Euclidean metric)

In [ ]:
inverse_errors = []

for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean', weights='distance')
    clf.fit(X_train_pca, y_train.ravel())
    y_pred = clf.predict(X_test_pca)
    inverse_errors.append(1 - accuracy_score(y_test, y_pred))
    print('Inverse: m =', best_m, 'k =', k, 'error =', inverse_errors[k-1])

In [ ]:
plt.plot(k_values, euclidean_errors, label='Euclidean Unweighted')
plt.plot(k_values, inverse_errors, label='Euclidean Inverse')
plt.legend()
plt.xlabel('k')
plt.ylabel('error')
plt.grid()
plt.title('Unweighted vs Inverse Euclidean Distance')
plt.show()

### 5. Implment the nearest local centroid classifier (with Euclidean distance) and apply it to the m-dimensional PCA reduced data for various values of k. 

Plot the test errors against k.

In [ ]:
h=0.2
cmap_light = plt.cm.Paired
cmap_bold = plt.cm.Paired

In [ ]:
k_values = range(1, 13)

errors = []

pca = PCA(n_components=best_m)  # Reduce to 2 dimensions for visualization
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# This algorithm is going to go through every training point and find the k nearest neighbors in each class of the training set
# Then, for each class, it will calculate the centroid of the k nearest neighbors. For each centroid it will calculate the distance
# to the test point. The predicted class is the one with the minimum distance.
'''
1. Isolate each class into a new variable
2. Find the 5 closest points in that class to the test point
3. Calculate the centroid of those 5 points
4. Store the distance from the centroid to the test point
5. Repeat for all classes
6. The class with the minimum distance is the predicted class
7. Store the predicted class
8. Repeat for all test points
9. Calculate the error
'''

clf = KNeighborsClassifier(n_neighbors=3, metric='euclidean', weights='distance')
clf.fit(X_train_pca, y_train.ravel())
clf.classes_ = np.unique(y_train)  # Ensure all classes are included
predicted_classes = []
for point in X_test_pca:
    distances = []
    for i in clf.classes_:
        X_class = X_train_pca[y_train.ravel() == i]
        if len(X_class) > 0:
            clf.fit(X_class, y_train[y_train.ravel() == i])
            # find the closest points in that class to the test point
            n_neighbors = min(5, len(X_class))
            five_closest = clf.kneighbors(point.reshape(1, -1), n_neighbors=n_neighbors, return_distance=False)
            centroid = np.mean(X_class[five_closest.flatten()], axis=0)

            # store the distance from the centroid to the test point
            distance = np.linalg.norm(centroid - point)
            distances.append(distance)
        else:
            distances.append(np.inf)  # If there are no samples in the class, set distance to infinity
    
    # the class with the minimum distance is the predicted class
    y_pred = clf.classes_[np.argmin(distances)]
    predicted_classes.append(y_pred)

errors.append(1 - accuracy_score(y_test, predicted_classes))
print('error =', errors[-1])

In [ ]:
for k in k_values:
    clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean', weights='distance')
    clf.fit(X_train_pca, y_train.ravel())
    
    # find the centroid of each class
    centroids = np.zeros((len(clf.classes_), best_m))
    for i in clf.classes_:
        centroids[i] = np.mean(X_train_pca[y_train.ravel() == i], axis=0)

    # For each test points, find the closest centroid
    y_pred = np.zeros(y_test.shape)
    for i in range(X_test_pca.shape[0]):
        distances = np.linalg.norm(centroids - X_test_pca[i], axis=1)
        y_pred[i] = np.argmin(distances)

    error = 1 - accuracy_score(y_test, y_pred)
    errors.append(error)
    print(f'k = {k}, error = {error*100:.2f}%')


In [ ]:
# Create a scatter plot of the training data in the PCA space
scatter = plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train.ravel(), cmap=cmap_bold, edgecolor='k', s=20)

# Create a custom legend
legend1 = plt.legend(*scatter.legend_elements(), title="Classes")
plt.gca().add_artist(legend1)

plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.title('Training data')
plt.show()

In [ ]:
plt.plot(k_values, errors)
plt.xlabel('k')
plt.ylabel('error')
plt.grid()
plt.title('Nearest Local Centroid')
plt.show()